I have found that certain scales in k-cdfs are not informative. This notebook will make copies of them that apply these scale cuts. Gotta do it for 

* Emus
* Sim Covmats
* Mock Measurements

In [21]:
import h5py
import numpy as np

In [22]:
fname = '/u/ki/swmclau2/des/Aemulators/knn_{k}_cdf_hsab_test/PearceKNN{k}pCDFHSABCosmoTest.hdf5'
new_fname = '/u/ki/swmclau2/des/Aemulators/knn_{k}_cdf_hsab_test/PearceKNN{k}pCDFCutHSABCosmoTest.hdf5'

In [23]:
# these computed where the average values - 1 std is less than 1/sqrt(n_ran)
keep_scales = {1: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False],
#2: [False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False],
2: [False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False],
4: [False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False],
#8: [False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, False, False, False, False]}
8: [False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, False, False, False, False]}

In [24]:
for k in [1,2,4,8]:
    print k
    with h5py.File(fname.format(k=k), 'r') as f, h5py.File(new_fname.format(k=k), 'w') as nf:
        for key in f.attrs:
            nf.attrs[key] = f.attrs[key]
        
        scales = np.array(keep_scales[k])
        # gotta add a bin 
        last_true = False
        last_true_idx = -1
        for i, c in enumerate(scales):
            if  c: 
                last_true = True
            if last_true and not c:
                last_true_idx = i
                break
        scales2 = list(scales)
        scales2.insert(last_true_idx, True)
        scales2 = np.array(scales2)

        nf.attrs['scale_bins'] = f.attrs['scale_bins'][scales2]
            
        for key in f:
            cdf =  f[key]['a_1.000']['obs'][()]
            cov =  f[key]['a_1.000']['cov'][()]

            cdf = cdf[:,scales]
            cov = cov[:,scales,:][:,:,scales]
            
            grp = nf.create_group(key+'/a_1.000/')
            obs_dset = grp.create_dataset('obs', data = cdf, compression='gzip')
            cov_dset = grp.create_dataset('cov', data = cov, compression='gzip')


1
2
4
8


In [5]:
cov_fname = '/u/ki/swmclau2/Git/pearce/bin/covmat/knn_%d_pcdf_sim_covmat.npy'
new_cov_fname = '/u/ki/swmclau2/Git/pearce/bin/covmat/knn_%d_pcdf_cut_sim_covmat.npy'

In [6]:
for k in [1,2,4,8]:
    scales = np.array(keep_scales[k])
    cov = np.load(cov_fname%k)
    
    cov = cov[scales, :][:, scales]
    np.save(new_cov_fname%k, cov)

In [7]:
cov_fname = '/u/ki/swmclau2/Git/pearce/bin/covmat/knn_pcdf_sim_covmat.npy'
new_cov_fname = '/u/ki/swmclau2/Git/pearce/bin/covmat/knn_pcdf_cut_sim_covmat.npy'

In [8]:
cov = np.load(cov_fname)
scales = np.hstack([np.array(keep_scales[k]) for k in [1,2,4,8]])
cov = cov[scales, :][:,scales]
np.save(new_cov_fname, cov)

In [9]:
meas_fname = '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_1_1_knn_%d_pcdf.npy'
new_meas_fname = '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_1_1_knn_%d_pcdf_cut.npy'

In [10]:
for k in [1,2,4,8]:
    obs = np.load(meas_fname%(k))
    print obs.shape
    obs = obs[np.array(keep_scales[k])]
    print obs.shape
    np.save(new_meas_fname%(k), obs)

(20,)
(15,)
(20,)
(13,)
(20,)
(11,)
(20,)
(9,)


In [11]:
meas_fname = '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_5_1_hsab_knn_%d_pcdf.npy'
new_meas_fname = '/u/ki/swmclau2/Git/pearce/bin/mock_measurements/testbox_5_1_hsab_knn_%d_cdf_pcut.npy'

In [12]:
for k in [1,2,4,8]:
    obs = np.load(meas_fname%(k))
    print obs.shape
    obs = obs[np.array(keep_scales[k])]
    print obs.shape
    np.save(new_meas_fname%(k), obs)

(20,)
(15,)
(20,)
(13,)
(20,)
(11,)
(20,)
(9,)
